In [1]:
from pathlib import Path
import os

yr = 2023
d = 16

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as np
def format_input(inp):
  c = 0
  arr = []
  cur_arr = []
  for i, r in enumerate(inp):
    c+= 1
    if r=='\n':
      c = 0
      arr.append(cur_arr)
      cur_arr = []
    else:
      cur_arr.append(r)
  arr.append(cur_arr)
  return np.array(arr, dtype=str)

In [3]:
from enum import Enum

# class syntax
class Directions(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4


class Beam:

  def __init__(self, loc, dir, arr):
    self.loc = loc
    self.dir = dir
    self.arr = arr
    self.done = False
    self.history = []


  def move_direction(self):
    next_x = self.loc[1]
    next_y = self.loc[0]
    if self.dir == Directions.UP:
      next_y = next_y-1
    elif self.dir == Directions.DOWN:
      next_y = next_y+1
    elif self.dir == Directions.LEFT:
      next_x = next_x-1
    elif self.dir == Directions.RIGHT:
      next_x = next_x+1
    else:
      raise Exception("Invalid Direction")

    self.loc = (next_y, next_x)


  def change_direction_on_mirror(self, cur_tile):
    mirror_dir_map = {
                    (Directions.LEFT, '\\'): Directions.UP,
                    (Directions.LEFT, '/'): Directions.DOWN,
                    (Directions.RIGHT, '\\'): Directions.DOWN,
                    (Directions.RIGHT, '/'): Directions.UP,
                    (Directions.UP, '\\'): Directions.LEFT,
                    (Directions.UP, '/'): Directions.RIGHT,
                    (Directions.DOWN, '\\'): Directions.RIGHT,
                    (Directions.DOWN, '/'): Directions.LEFT
                      }
    if (self.dir, cur_tile) not in mirror_dir_map:
      raise Exception("ILLEGAL MIRROR COMBO: ", (self.dir, cur_tile))
    else:
      self.dir = mirror_dir_map[(self.dir, cur_tile)]


  def is_on_contraption(self):
    contraption_height = self.arr.shape[0]
    contraption_width = self.arr.shape[1]
    return ((0 <= self.loc[0] and self.loc[0] < contraption_height)
          and (0 <= self.loc[1] and self.loc[1] < contraption_width))


  def next(self):

    if not self.is_on_contraption():
      self.done = True

    out = []
    if not self.done:
      self.history.append((self.loc, self.dir))
      cur_tile = self.arr[self.loc]

      if cur_tile in ['\\', '/']:
        self.change_direction_on_mirror(cur_tile)
        self.move_direction()
      elif cur_tile=='-' and self.dir in [Directions.UP, Directions.DOWN]:
        out = [Beam(self.loc, Directions.LEFT, self.arr), Beam(self.loc, Directions.RIGHT, self.arr)]
        self.done = True
      elif cur_tile=='|' and self.dir in [Directions.RIGHT, Directions.LEFT]:
        out = [Beam(self.loc, Directions.UP, self.arr), Beam(self.loc, Directions.DOWN, self.arr)]
        self.done = True
      elif (cur_tile == '.'
        or (cur_tile == '-'
            and self.dir in [Directions.LEFT, Directions.RIGHT])
        or (cur_tile == '|'
            and self.dir in [Directions.UP, Directions.DOWN])
          ):
        self.move_direction()
    return out

In [4]:
def count_energized_tiles(formatted_input, start_beam_info=((0,0), Directions.RIGHT)):
  '''
  Start with one beam in the upper left corner.
  At each timestep, advance all beams to their next respective states
  Beams terminate when:
    They hit a mirror (and add two child beams to the list of beams)
    OR They leave the bounds of the contraption
    OR They reach a state that they were already at (same location and direction)

  While there are non-terminated beams, continue stepping through time.

  Each beam keeps a history of all the location/direction combos it has visited.

  At the end of the iteration, count the unique number of visited spaces across all beams.
  '''
  beams = [Beam(*start_beam_info, formatted_input)]
  cnt = 0
  full_history = {}
  while not(all([b.done for b in beams])):
    new_beams = []
    for b in beams:
      children = b.next()
      new_beams.append(b)
      for c in children:
        new_beams.append(c)
    beams = new_beams
    to_remove = []
    for b in beams:
      if len(b.history)!=0 and b.history[-1] in full_history:
        b.done = True
        to_remove.append(b)
      for h in b.history:
        full_history[h]=True
    for b in reversed(to_remove):
      beams.remove(b)
    cnt+=1
  return len(set([x[0] for x in full_history.keys()]))

In [5]:
def max_energized_tiles(formatted_input):
  '''
  Just find all the possible beam starts along the edge,
  run count_energized_tiles() on each, and return the max
  '''
    
  from tqdm.notebook import tqdm
  from multiprocessing import Pool

  l, w = formatted_input.shape
  top_edge = [(0, i) for i in range(w)]
  left_edge = [(i,0) for i in range(1,l-1)]
  right_edge = [(i,w-1) for i in range(1,l-1)]
  bottom_edge = [(l-1,i) for i in range(w)]

  start_beam_infos = (list(zip(top_edge, [Directions.DOWN]*w))
                + list(zip(left_edge, [Directions.RIGHT]*l))
                + list(zip(right_edge, [Directions.LEFT]*l))
                + list(zip(bottom_edge, [Directions.UP]*w)))

  n_energized = []
  for s in tqdm(start_beam_infos):
    n_energized.append(count_energized_tiles(formatted_input, s))

  return max(n_energized)


In [6]:
import time

t = time.time()

formatted_input = format_input(inp)

print(count_energized_tiles(formatted_input))
print(max_energized_tiles(formatted_input))

print('\nRUNTIME: ', time.time()-t)

7199


  0%|          | 0/436 [00:00<?, ?it/s]

7438

RUNTIME:  17.72377038002014
